In [3]:
!pip install typer

In [6]:
!pip install spacy

In [3]:
import json

import typer
from pathlib import Path

from spacy.tokens import Span, DocBin, Doc
from spacy.vocab import Vocab
from wasabi import Printer
from spacy.tokenizer import Tokenizer
from spacy.lang.en import English
from spacy.util import compile_infix_regex
import re
import spacy

nlp = spacy.blank("en")
# Create a blank Tokenizer with just the English vocab

msg = Printer()

SYMM_LABELS = ["Binds"]
MAP_LABELS = {
    "ASP-OPI": "ASP-OPI"
}

ann = "rajdeep_json_ubiai.json"
train_file='rajdeep_json_ubiai.spacy'
dev_file='suvam.spacy'
test_file='baitalik.spacy'

def main(json_loc: Path, train_file: Path, dev_file: Path, test_file: Path):
    """Creating the corpus from the Prodigy annotations."""
    Doc.set_extension("rel", default={},force=True)
    vocab = Vocab()

    docs = {"train": [], "dev": [], "test": [], "total": []}
    ids = {"train": set(), "dev": set(), "test": set(), "total":set()}
    count_all = {"train": 0, "dev": 0, "test": 0,"total": 0}
    count_pos = {"train": 0, "dev": 0, "test": 0,"total": 0}

    with open(json_loc, encoding="utf8") as jsonfile:
        file = json.load(jsonfile)
        for example in file:
            span_starts = set()
            neg = 0
            pos = 0
                    # Parse the tokens
            tokens=nlp(example["document"])    

            spaces=[]
            spaces = [True if tok.whitespace_ else False for tok in tokens]
            words = [t.text for t in tokens]
            doc = Doc(nlp.vocab, words=words, spaces=spaces)


            # Parse the GGP entities
            spans = example["tokens"]
            entities = []
            span_end_to_start = {}
            for span in spans:
                entity = doc.char_span(
                     span["start"], span["end"], label=span["entityLabel"]
                 )


                span_end_to_start[span["token_start"]] = span["token_start"]
                #print(span_end_to_start)
                entities.append(entity)
                span_starts.add(span["token_start"])

            doc.ents = entities

            # Parse the relations
            rels = {}
            for x1 in span_starts:
                for x2 in span_starts:
                    rels[(x1, x2)] = {}
                    #print(rels)
            relations = example["relations"]
            #print(len(relations))
            for relation in relations:
                # the 'head' and 'child' annotations refer to the end token in the span
                # but we want the first token
                start = span_end_to_start[relation["head"]]
                end = span_end_to_start[relation["child"]]
                label = relation["relationLabel"]
                #print(rels[(start, end)])
                #print(label)
                #label = MAP_LABELS[label]
                if label not in rels[(start, end)]:
                    rels[(start, end)][label] = 1.0
                    pos += 1
                    #print(pos)
                    #print(rels[(start, end)])

            # The annotation is complete, so fill in zero's where the data is missing
            for x1 in span_starts:
                for x2 in span_starts:
                    for label in MAP_LABELS.values():
                        if label not in rels[(x1, x2)]:
                            neg += 1
                            rels[(x1, x2)][label] = 0.0

                            #print(rels[(x1, x2)])
            doc._.rel = rels
            #print(doc._.rel)

            # only keeping documents with at least 1 positive case
            if pos > 0:
                    docs["total"].append(doc)
                    count_pos["total"] += pos
                    count_all["total"] += pos + neg

                    
                    
    print(len(docs["total"]))
    docbin = DocBin(docs=docs["total"], store_user_data=True)
    docbin.to_disk(train_file)
    msg.info(
        f"{len(docs['total'])} training sentences"
    )


main(ann, train_file, dev_file, test_file)


TypeError: object of type 'NoneType' has no len()

In [4]:
with open("rajdeep_json_ubiai.json", encoding="utf8") as jsonfile:
        file = json.load(jsonfile)

In [5]:
file

[{'document': "Good battery back up.Decent Display.Fast fingerprint sensor.Rear camera quality is good.front camera is not up to the mark.90 Hz refresh rate is good.Good user interface.sound quality is low comparing with other smartphones in same price segment.Camera , battery back up , display , are awesome 👌.This smartphone is very good in camera and battery And value for money from Samsung with clean and smooth Ui and good battery life alsoRear and front cameras are superb.6000 mah battery is just wow.Pros:- Battery backup is simply awesome.I got 8+ hours SOT Performance is way better than Xiaomi and realme at this range.No security risks Camera is good.big battery so takes 3 hrs to charge!Best mobile for in this price range, Battery backup is highlight.Camera is good battery is good fingure unlock sensor is very good and fast it is very thick due to large battery this phone is not for slim phone lovers but phone is very good and price are also good , good job samsungIt's real good 

In [8]:
for example in file:
    print(example["tokens"])
    break

[{'text': 'Display', 'start': 29, 'end': 36, 'token_start': 7, 'token_end': 7, 'entityLabel': 'ASP'}, {'text': 'Decent', 'start': 22, 'end': 28, 'token_start': 6, 'token_end': 6, 'entityLabel': 'OPI'}, {'text': 'fingerprint sensor', 'start': 43, 'end': 61, 'token_start': 11, 'token_end': 12, 'entityLabel': 'ASP'}, {'text': 'Fast', 'start': 38, 'end': 42, 'token_start': 10, 'token_end': 10, 'entityLabel': 'OPI'}, {'text': 'Rear camera', 'start': 63, 'end': 74, 'token_start': 15, 'token_end': 16, 'entityLabel': 'ASP'}, {'text': 'good', 'start': 86, 'end': 90, 'token_start': 19, 'token_end': 19, 'entityLabel': 'OPI'}, {'text': 'front camera', 'start': 92, 'end': 104, 'token_start': 22, 'token_end': 23, 'entityLabel': 'ASP'}, {'text': 'not up to the mark', 'start': 108, 'end': 126, 'token_start': 25, 'token_end': 29, 'entityLabel': 'OPI'}, {'text': '90 Hz refresh rate', 'start': 128, 'end': 146, 'token_start': 32, 'token_end': 35, 'entityLabel': 'ASP'}, {'text': 'good', 'start': 150, 'end'